In [ ]:
# import torch
# from transformers import AutoModelForSequenceClassification, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
# model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
# model.eval()

# pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
# with torch.no_grad():
#     inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=1024)
#     scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    

In [1]:
import requests
from openai import OpenAI
import pandas as pd
import re

In [2]:
import pickle

with open('voyage.pkl', 'rb') as f:
    data = pickle.load(f)

In [17]:
data[-1]

{'page': 521,
 'title': 'Ii Topics In Pure Mathematics/12 Additional Topics/12.2 Inductively Defined Sets',
 'content': '<markdown>\r\n### Theorem 12.2.38 (Structural induction principle for quotient-inductive sets)\r\n\r\nLet \\( Q \\) be a quotient-inductive set and let \\( p(x) \\) be a logical formula with free variable \\( x \\in Q \\). If for all rules \\( \\sigma \\) and all \\( x_1, \\ldots, x_r \\in Q \\) we have\r\n\r\n\\[\r\n[\\forall i \\in [r], \\, p(x_i)] \\Rightarrow p(f_\\sigma(x_1, x_2, \\ldots, x_r))\r\n\\]\r\n\r\nthen \\( p(x) \\) is true for all \\( x \\in Q \\).\r\n\r\n**Proof**\r\n\r\nLet \\( q : A \\rightarrow Q \\) be the quotient map from the inductively defined set \\( A \\), and let \\( \\overline{p}(x) \\) be the logical formula with free variable \\( x \\in A \\) defined by letting \\( \\overline{p}(x) \\) mean \\( p(q(x)) \\).\r\n\r\nSince \\( q \\) is surjective, if \\( \\overline{p}(x) \\) is true for all \\( x \\in A \\), then \\( p(x) \\) is true for a

In [ ]:
# Reranking
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
model.eval()

# pairs = [[x["Q"], x["A"]] for x in data]
# with torch.no_grad():
#     inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
#     scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
#     print(scores)

tensor([-5.6085,  5.7623])


In [21]:
def sort_by_another_list(a, b):
    """
    Sorts list 'a' based on the values of list 'b'.
    :param a: List of floats.
    :param b: List of floats (same length as 'a').
    :return: List 'a' sorted based on 'b'.
    """
    if len(a) != len(b):
        raise ValueError("Both lists must have the same length")
    
    # Pair elements from both lists and sort based on 'b'
    sorted_pairs = sorted(zip(b, a), reverse=True)
    
    # Extract the sorted 'a' values
    sorted_a = [val[1] for val in sorted_pairs]
    
    return sorted_a

In [ ]:
top_k = 10
top1 = 0
top3 = 0
top5 = 0
top10 = 0
top20 = 0
top50 = 0
top100 = 0

for d in data:
    d = data[6]
    query = d['Q']
    answer = d['A']
    page_number = d['page']
    retrived_pages = []
    retrived_indexes = d['question_retrived_page_rank'][:top_k]
    retrived_indexes = list(set(sorted([list(retrived_indexes[u].keys())[0] for u in range(len(retrived_indexes))])))
    print(retrived_indexes)
    for x in data:
        if(int(x['page']) in retrived_indexes):
            retrived_pages.append(x['content'])
            print(x['page'])

    retrived_pages = list(set(retrived_pages))
    pairs = [[query, page] for page in retrived_pages]
    with torch.no_grad():
        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt')
        scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
        print(scores)
        print(retrived_indexes)
        reranked_retrieved_indexes = sort_by_another_list(retrived_indexes, scores)
        print(reranked_retrieved_indexes)
    
    if page_number in reranked_retrieved_indexes[:1]: top1 += 1
    if page_number in reranked_retrieved_indexes[:3]: top3 += 1
    if page_number in reranked_retrieved_indexes[:5]: top5 += 1
    if page_number in reranked_retrieved_indexes[:10]: top10 += 1
    if page_number in reranked_retrieved_indexes[:20]: top20 += 1
    if page_number in reranked_retrieved_indexes[:50]: top50 += 1
    if page_number in reranked_retrieved_indexes[:100]: top100 += 1
    print(page_number)

23
24
25
278
301
302
309
301
302
309
347
['<markdown>\r\n- **(IS)** Fix \\( n \\geq 0 \\) and suppose that the requirements of the theorem are satisfied for all the natural numbers up to and including \\( n \\).\r\n\r\nBy the division theorem ([Theorem 7.1.1](#)), there exist unique \\( u, v \\in \\mathbb{N} \\) such that\r\n\r\n\\[\r\nn + 1 = ub + v \\quad \\text{and} \\quad v \\in \\{0, 1, \\ldots, b - 1\\}\r\n\\]\r\n\r\nSince \\( b \\geq 2 \\), we have \\( u < n + 1 \\), and so \\( u \\leq n \\). It follows from the induction hypothesis that there exist unique \\( r \\in \\mathbb{N} \\) and \\( d_1, \\ldots, d_r \\in \\{0, 1, \\ldots, b - 1\\} \\) such that\r\n\r\n\\[\r\nu = \\sum_{i=0}^{r} d_{i+1} b^i\r\n\\]\r\n\r\nand \\( d_r \\neq 0 \\). Writing \\( d_0 = v \\) yields\r\n\r\n\\[\r\nn = ub + v = \\sum_{i=0}^{r} d_{i+1} b^{i+1} + d_0 = \\sum_{i=0}^{r} d_i b^i\r\n\\]\r\n\r\nSince \\( d_r \\neq 0 \\), this proves existence.\r\n\r\nFor uniqueness, suppose that there exists \\( s \\in 

tensor([-8.8954, -4.5130, -7.1284, -7.5792,  2.7106, -9.4424, -7.8958, -9.0361])
[23, 24, 25, 278, 301, 301, 302, 309, 309, 347]


ValueError: Both lists must have the same length

In [8]:
print(top1/477)
print(top3/477)
print(top5/477)
print(top10/477)

0.5052410901467506
0.8406708595387841
0.9035639412997903
0.9832285115303984
